In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph

In [2]:
from typing_extensions import TypedDict
from typing import Literal
from typing import Annotated
from operator import add

class State(TypedDict):
    foo: Annotated[list[int], add]

In [3]:
def node_1(state):
    print("---Node 1---")
    return { "foo": [state["foo"][0] + 1]}




In [4]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("node_1", node_1)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

graph = builder.compile()

In [5]:
graph.invoke({"foo" : [1]})

---Node 1---


{'foo': [1, 2]}

In [6]:
def node_1(state):
    print("---Node 1---")
    return { "foo": [state["foo"][-1] + 1]}

def node_2(state):
    print("---Node 2---")
    return { "foo": [state["foo"][-1] + 1]}

def node_3(state):
    print("---Node 3---")
    return { "foo": [state["foo"][-1] + 1]}


builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_1", "node_3")
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

graph = builder.compile()

graph.invoke({"foo" : [1]})


---Node 1---
---Node 2---
---Node 3---


{'foo': [1, 2, 3, 3]}

## Custom Reducers

In [7]:
def reduce_list(left: list | None, right : list | None) ->list:
    """
    Safely Combine two list
    
    Args:
        left (list | None)
        right (list | None)
    
    Returns:
        list: A new list containing all the elements
    """
    
    if not left:
        left = []
        
    if not right:
        right = []
        
    return left + right

In [8]:
class DefaultState(TypedDict):
    foo: Annotated[list[int], add]

class CustomReducerState(TypedDict):
    foo: Annotated[list[int], reduce_list]

In [11]:
def node_1(state):
    print("---Node 1---")
    return {"foo": [2]}

builder = StateGraph(DefaultState)
builder.add_node("node_1", node_1)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

graph = builder.compile()

try:
    print(graph.invoke({"foo": None}))
except TypeError as e:
    print(f"TypeError Occured: {e}")


TypeError Occured: can only concatenate list (not "NoneType") to list


In [12]:
def node_1(state):
    print("---Node 1---")
    return {"foo": [2]}

builder = StateGraph(CustomReducerState)
builder.add_node("node_1", node_1)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

graph = builder.compile()

try:
    print(graph.invoke({"foo": None}))
except TypeError as e:
    print(f"TypeError Occured: {e}")


---Node 1---
{'foo': [2]}
